In [153]:
import glob
import json
import pandas as pd

In [154]:
tweets_dict = {}

# Read every *.jsons file in the ./tweet_jsons/ directory and copy them into a pandas dataframe
for filepath in glob.iglob('./tweet_jsons/*.json'):
    f = open(filepath, "r")
    tweets_df = pd.read_json(filepath, lines=True)
    tweets_dict[filepath[14:-5]] = tweets_df
    
print(tweets_dict.keys())

dict_keys(['pmarca', 'novogratz', 'rogerkver', 'needacoin', 'crypto_rand', 'ToneVays', 'legendofcrypto', 'CryptoChoe', 'cryptoBully', 'VitalikButerin', 'peterlbrandt', 'NicTrades', 'DiaryofaMademan', 'crypto_God', 'tuurdemeester', 'APompliano', 'lopp', 'nickszabo4', 'IOHK_Charles', 'CryptoCobain', '6BillionPeople', 'cryptodemedici', 'CryptoOrca', 'VinnyLingham', 'WhalePanda', 'cryptopathic', 'IAMJOSEPHYYOUNG', 'CryptoHustle', 'gavinandresen', 'Crypto_goat_', 'PhilakoneCrypt', 'aantonop', 'brian_armstrong', 'cryptobanger', '22loops', 'CryptoYoda1338', 'VentureCoinist', 'secretsofcrypto', 'barrysilbert', 'Thecryptodog', 'onemanatatime', 'cryptomocho', 'officialmcafee', 'erikvoorhees', 'IOTA_Charles', 'thecryptomofo', 'cryptogat', 'balajis', 'ZeusZissou', 'CremeDeLaCrypto', 'CryptoTutor', 'cz_binance', 'jimmysong', 'justinsuntron', 'angelabtc', 'notsofast', 'charlieshrem', 'SatoshiLite'])


In [155]:
#Keyword filtering: only retain tweets related to cryptocurrency and remove empty lists

keywords = ["btc", "ethereum", "coin", "crypto", "hodl"]

for key in list(tweets_dict.keys()):
    #print(tweets_dict[key])
    if not tweets_dict[key].empty:
        tweets_dict[key] = tweets_dict[key][tweets_dict[key]["content"].str.contains('|'.join(keywords), na = False, case = False)]
    if tweets_dict[key].empty:
        del tweets_dict[key]

In [158]:
print(tweets_dict.keys())

dict_keys(['novogratz', 'rogerkver', 'needacoin', 'crypto_rand', 'ToneVays', 'VitalikButerin', 'peterlbrandt', 'NicTrades', 'DiaryofaMademan', 'APompliano', 'lopp', 'nickszabo4', 'IOHK_Charles', 'CryptoCobain', '6BillionPeople', 'VinnyLingham', 'WhalePanda', 'cryptopathic', 'CryptoHustle', 'gavinandresen', 'Crypto_goat_', 'aantonop', 'brian_armstrong', 'cryptobanger', '22loops', 'CryptoYoda1338', 'VentureCoinist', 'secretsofcrypto', 'barrysilbert', 'Thecryptodog', 'onemanatatime', 'cryptomocho', 'officialmcafee', 'erikvoorhees', 'balajis', 'ZeusZissou', 'CremeDeLaCrypto', 'CryptoTutor', 'cz_binance', 'jimmysong', 'justinsuntron', 'notsofast', 'charlieshrem', 'SatoshiLite'])
